In [17]:
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import pandas as pd
import os
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pickle

In [18]:
rent = "final.csv"
rent_df = pd.read_csv(rent)
rent_df = rent_df.drop(["Unnamed: 0"], axis = 1)
rent_df.head()

,Property_Code,Tenant_Code,Event,DateOccurred,Unit_Code,DateMoveIn,DateMoveOut,Rent,Vacancy,SQFT,BedRooms,Bathroom,chrent,LengthStay
0,ce,t0013745,Rent Change,4/30/2013,C05266,11/9/2006,NaN,870.0,0,1400,3,2.5,870.0,NaN
1,ce,t0013745,Rent Change,5/31/2014,C05266,11/9/2006,1/31/2015,896.0,1,1400,3,2.5,26.0,245 days 00:00:00.000000000
2,ce,t0013747,Rent Change,4/30/2013,C05270,7/16/2012,8/11/2013,770.0,1,1088,2,1.5,770.0,103 days 00:00:00.000000000
3,ce,t0013748,Rent Change,4/30/2013,C05272,9/28/2012,NaN,770.0,0,1088,2,1.5,770.0,NaN
4,ce,t0013748,Rent Change,9/30/2013,C05272,9/28/2012,NaN,800.0,0,1088,2,1.5,30.0,NaN


In [19]:
indexNames = rent_df[rent_df["Rent"] == rent_df ["chrent"]].index
rent_df.drop(indexNames, inplace = True)
outliers = rent_df[rent_df["chrent"] >= 500].index
rent_df.drop(outliers, inplace = True)
outliers2 = rent_df[rent_df["chrent"] <= -500].index
rent_df.drop(outliers2, inplace = True)
rent_df

,Property_Code,Tenant_Code,Event,DateOccurred,Unit_Code,DateMoveIn,DateMoveOut,Rent,Vacancy,SQFT,BedRooms,Bathroom,chrent,LengthStay
1,ce,t0013745,Rent Change,5/31/2014,C05266,11/9/2006,1/31/2015,896.0,1,1400,3,2.5,26.0,245 days 00:00:00.000000000
4,ce,t0013748,Rent Change,9/30/2013,C05272,9/28/2012,NaN,800.0,0,1088,2,1.5,30.0,NaN
5,ce,t0013748,Rent Change,11/1/2014,C05272,9/28/2012,10/31/2015,824.0,1,1088,2,1.5,24.0,364 days 00:00:00.000000000
9,ce,t0013749,Rent Change,12/31/2013,C05274,11/26/2008,NaN,915.0,0,1400,3,2.5,35.0,NaN
10,ce,t0013749,Rent Change,3/1/2015,C05274,11/26/2008,NaN,924.0,0,1400,3,2.5,9.0,NaN
11,ce,t0013749,Rent Change,2/29/2016,C05274,11/26/2008,NaN,970.0,0,1400,3,2.5,46.0,NaN
12,ce,t0013749,Rent Change,2/28/2018,C05274,11/26/2008,NaN,1044.0,0,1400,3,2.5,74.0,NaN
14,ce,t0013750,Rent Change,6/30/2013,C05276,7/21/2001,NaN,890.0,0,1400,3,2.5,10.0,NaN
15,ce,t0013750,Rent Change,6/30/2014,C05276,7/21/2001,NaN,917.0,0,1400,3,2.5,27.0,NaN
16,ce,t0013750,Rent Change,6/30/2015,C05276,7/21/2001,NaN,945.0,0,1400,3,2.5,28.0,NaN


In [20]:
X = rent_df[["SQFT", "BedRooms", "Bathroom", "chrent", "Property_Code"]]
y = rent_df["Vacancy"]
print(X.shape, y.shape)

(6215, 5) (6215,)


In [21]:
data = X.copy()

data_binary_encoded = pd.get_dummies(data, columns = ["Property_Code"])
data_binary_encoded.head()

,SQFT,BedRooms,Bathroom,chrent,Property_Code_ce,Property_Code_fh,Property_Code_mpg,Property_Code_sh
1,1400,3,2.5,26.0,1,0,0,0
4,1088,2,1.5,30.0,1,0,0,0
5,1088,2,1.5,24.0,1,0,0,0
9,1400,3,2.5,35.0,1,0,0,0
10,1400,3,2.5,9.0,1,0,0,0


In [22]:
data_binary_encoded = data_binary_encoded.fillna(0)

In [23]:
data_binary_encoded.dtypes

SQFT                   int64
BedRooms               int64
Bathroom             float64
chrent               float64
Property_Code_ce       uint8
Property_Code_fh       uint8
Property_Code_mpg      uint8
Property_Code_sh       uint8
dtype: object

In [24]:
data_binary_encoded ["BedRooms"]= data_binary_encoded["BedRooms"].astype(float)
data_binary_encoded ["Property_Code_ce"]= data_binary_encoded["Property_Code_ce"].astype(float)
data_binary_encoded ["Property_Code_fh"]= data_binary_encoded["Property_Code_fh"].astype(float)
data_binary_encoded ["Property_Code_mpg"]= data_binary_encoded["Property_Code_mpg"].astype(float)
data_binary_encoded ["Property_Code_sh"]= data_binary_encoded["Property_Code_sh"].astype(float)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_binary_encoded, y, stratify=y, random_state=42)

In [26]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7232947232947233

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7245817245817245

In [28]:
prediction = rf.predict(X_test)
prediction_df =pd.DataFrame({"Predictions": prediction, "Actual": y_test})
prediction_df.loc[prediction_df["Predictions"] == 1, "Predictions"] = "Vacant"
prediction_df.loc[prediction_df["Predictions"] == 0, "Predictions"] = "Occupied"
prediction_df.loc[prediction_df["Actual"] == 1, "Actual"] = "Vacant"
prediction_df.loc[prediction_df["Actual"] == 0, "Actual"] = "Occupied"
prediction_df.head()

,Predictions,Actual
2235,Occupied,Occupied
4923,Occupied,Occupied
4772,Occupied,Occupied
5085,Occupied,Occupied
3665,Occupied,Occupied


In [29]:
prediction_df.loc[prediction_df["Predictions"] == "Vacant"] 

,Predictions,Actual
4429,Vacant,Vacant
4641,Vacant,Occupied
3812,Vacant,Vacant
2420,Vacant,Occupied
988,Vacant,Occupied
146,Vacant,Vacant
1333,Vacant,Occupied
2431,Vacant,Occupied
4408,Vacant,Vacant
5520,Vacant,Occupied


In [30]:
feature_names = data_binary_encoded.columns
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.8036964467006937, 'chrent'),
 (0.09197747691913509, 'SQFT'),
 (0.041547789430155205, 'Bathroom'),
 (0.01836821200327742, 'BedRooms'),
 (0.01479909428643259, 'Property_Code_mpg'),
 (0.013060942022781163, 'Property_Code_fh'),
 (0.009123229402043128, 'Property_Code_ce'),
 (0.007426809235481683, 'Property_Code_sh')]

In [31]:
filename = "logisitic_depp_network.pkl"
list_pickle = open(filename, "wb")
pickle.dump(rf, list_pickle)
list_pickle.close()